# 1. 윤석열

In [13]:
#1
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "윤석열_한겨레_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 


# 기사 내용을 추출하는 함수
def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        content = ""

        # 1. <p> 태그 (class="text")에서 텍스트 추출
        article_div = soup.find("div", class_="article-text")
        if article_div:
            paragraphs = article_div.find_all("p", class_="text")
            content += "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 2. <span> 태그에서 추가 텍스트 추출
            spans = article_div.find_all("span")
            content += "\n" + "\n".join([span.get_text(strip=True) for span in spans])

            # 3. 기타 <div> 태그에서 추가 텍스트 추출
            divs = article_div.find_all("div", recursive=False)
            content += "\n".join([div.get_text(strip=True) for div in divs])

        # 5. <div class="arti-txt ">에서 텍스트 추출
        arti_txt_div = soup.find("div", class_="arti-txt")
        if arti_txt_div:
            # <p> 태그에서 텍스트 추출
            paragraps = arti_txt_div.find_all("p")
            content += "\n".join([k.get_text(strip=True) for k in paragraps])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없는 경우 처리
        if not content:
            print(f"본문 텍스트를 찾을 수 없습니다: {url}")
            return None

        return content

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None


# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
for idx, row in df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Hankyoreh_YSY_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")

    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (1/200): https://www.hani.co.kr/arti/politics/politics_general/1033672.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200\Hankyoreh_YSY_1_200.txt
크롤링 중 (2/200): https://www.hani.co.kr/arti/opinion/column/1033851.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200\Hankyoreh_YSY_2_200.txt
크롤링 중 (3/200): https://www.hani.co.kr/arti/politics/assembly/1033304.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200\Hankyoreh_YSY_3_200.txt
크롤링 중 (4/200): https://www.hani.co.kr/arti/politics/politics_general/1034103.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200\Hankyoreh_YSY_4_200.txt
크롤링 중 (5/200): https://www.hani.co.kr/arti/politics/politics_general/1031025.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200\Hankyoreh_YSY_5_200.txt
크롤링 중 (6/200): https://www.hani.co.kr/arti/politics/politics_general/1031153.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200\Hankyoreh_YSY_6_200.txt
크롤링 중 (7/200): https://www.hani.co.kr/arti/politics/p

In [35]:
# 빠뜨린 기사들 스크랩_윤석열
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "윤석열_한겨레_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\한겨레\윤석열 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 

# 기사 내용을 추출하는 함수
def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        content = ""

        # 1. <p> 태그 (class="text")에서 텍스트 추출
        article_div = soup.find("div", class_="article-text")
        if article_div:
            paragraphs = article_div.find_all("p", class_="text")
            content += "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 2. <span> 태그에서 추가 텍스트 추출
            spans = article_div.find_all("span")
            content += "\n" + "\n".join([span.get_text(strip=True) for span in spans])

            # 3. 기타 <div> 태그에서 추가 텍스트 추출
            divs = article_div.find_all("div", recursive=False)
            content += "\n".join([div.get_text(strip=True) for div in divs])

        # 5. <div class="arti-txt ">에서 텍스트 추출
        arti_txt_div = soup.find("div", class_="arti-txt")
        if arti_txt_div:
            # <p> 태그에서 텍스트 추출
            paragraps = arti_txt_div.find_all("p")
            content += "\n".join([k.get_text(strip=True) for k in paragraps])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없는 경우 처리
        if not content:
            print(f"본문 텍스트를 찾을 수 없습니다: {url}")
            return None

        return content

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
# 크롤링할 특정 행 번호 리스트 (1-based index)
selected_rows = [189]

# 0-based 인덱스로 변환 (Python 인덱스는 0부터 시작)
selected_rows = [row - 1 for row in selected_rows]

# 데이터프레임에서 해당 행 번호만 필터링
filtered_df = df.iloc[selected_rows]

# 크롤링 및 저장
for idx, row in filtered_df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Hankyoreh_YSY_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")


    # 요청 간 간격 두기
    time.sleep(3)

# WebDriver 종료
driver.quit()


크롤링 중 (189/200): https://www.hani.co.kr/arti/opinion/column/1032728.html
본문 텍스트를 찾을 수 없습니다: https://www.hani.co.kr/arti/opinion/column/1032728.html
본문이 비어 있습니다: https://www.hani.co.kr/arti/opinion/column/1032728.html


# 2. 이재명

In [36]:
#2
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "이재명_한겨레_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\한겨레\이재명 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 


# 기사 내용을 추출하는 함수
def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        content = ""

        # 1. <p> 태그 (class="text")에서 텍스트 추출
        article_div = soup.find("div", class_="article-text")
        if article_div:
            paragraphs = article_div.find_all("p", class_="text")
            content += "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 2. <span> 태그에서 추가 텍스트 추출
            spans = article_div.find_all("span")
            content += "\n" + "\n".join([span.get_text(strip=True) for span in spans])

            # 3. 기타 <div> 태그에서 추가 텍스트 추출
            divs = article_div.find_all("div", recursive=False)
            content += "\n".join([div.get_text(strip=True) for div in divs])

        # 5. <div class="arti-txt ">에서 텍스트 추출
        arti_txt_div = soup.find("div", class_="arti-txt")
        if arti_txt_div:
            # <p> 태그에서 텍스트 추출
            paragraps = arti_txt_div.find_all("p")
            content += "\n".join([k.get_text(strip=True) for k in paragraps])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없는 경우 처리
        if not content:
            print(f"본문 텍스트를 찾을 수 없습니다: {url}")
            return None

        return content

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None


# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
for idx, row in df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Hankyoreh_LJM_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")

    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (1/200): https://www.hani.co.kr/arti/politics/politics_general/1033672.html


KeyboardInterrupt: 

In [37]:
# 빠뜨린 기사들 스크랩_이재명
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "이재명_한겨레_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\한겨레\이재명 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 


# 기사 내용을 추출하는 함수
def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        content = ""

        # 1. <p> 태그 (class="text")에서 텍스트 추출
        article_div = soup.find("div", class_="article-text")
        if article_div:
            paragraphs = article_div.find_all("p", class_="text")
            content += "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 2. <span> 태그에서 추가 텍스트 추출
            spans = article_div.find_all("span")
            content += "\n" + "\n".join([span.get_text(strip=True) for span in spans])

            # 3. 기타 <div> 태그에서 추가 텍스트 추출
            divs = article_div.find_all("div", recursive=False)
            content += "\n".join([div.get_text(strip=True) for div in divs])

        # 5. <div class="arti-txt ">에서 텍스트 추출
        arti_txt_div = soup.find("div", class_="arti-txt")
        if arti_txt_div:
            # <p> 태그에서 텍스트 추출
            paragraps = arti_txt_div.find_all("p")
            content += "\n".join([k.get_text(strip=True) for k in paragraps])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없는 경우 처리
        if not content:
            print(f"본문 텍스트를 찾을 수 없습니다: {url}")
            return None

        return content

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
# 크롤링할 특정 행 번호 리스트 (1-based index)
selected_rows = [64, 68, 75, 173, 189]

# 0-based 인덱스로 변환 (Python 인덱스는 0부터 시작)
selected_rows = [row - 1 for row in selected_rows]

# 데이터프레임에서 해당 행 번호만 필터링
filtered_df = df.iloc[selected_rows]

# 크롤링 및 저장
for idx, row in filtered_df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Hankyoreh_LJM_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")


    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (64/200): https://h21.hani.co.kr/arti/politics/politics_general/51641.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\이재명 기사 200\Hankyoreh_LJM_64_200.txt
크롤링 중 (68/200): https://h21.hani.co.kr/arti/society/society_general/51678.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\이재명 기사 200\Hankyoreh_LJM_68_200.txt
크롤링 중 (75/200): https://h21.hani.co.kr/arti/politics/politics_general/51673.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\이재명 기사 200\Hankyoreh_LJM_75_200.txt
크롤링 중 (173/200): https://h21.hani.co.kr/arti/politics/politics_general/51616.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\이재명 기사 200\Hankyoreh_LJM_173_200.txt
크롤링 중 (189/200): https://h21.hani.co.kr/arti/politics/politics_general/51682.html
기사 저장 완료: C:\Users\Administrator\2024-2\한겨레\이재명 기사 200\Hankyoreh_LJM_189_200.txt
